In [1]:
%matplotlib inline
import numpy as np
import QuantLib as ql

In [2]:
today = ql.Date(7, ql.March, 2023)
ql.Settings.instance().evaluationDate = today

## The instrument  
As a sample instrument, we’ll take a textbook example: a European option.

In [3]:
option = ql.EuropeanOption(
    ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
    ql.EuropeanExercise(ql.Date(7, ql.June, 2023))
)

## First pricing method: analytic Black-Scholes formula  
The different pricing methods are implemented as pricing engines holding the required market data.  
The first we’ll use is the one encapsulating the analytic Black-Scholes formula.  
First, we collect the quoted market data. We’ll assume flat risk-free rate and volatility, so they can
be expressed by **SimpleQuote** instances: those model numbers whose value can change and that
can notify observers when this happens. The underlying value is at 100, the risk-free value at 1%,
and the volatility at 20%.

In [4]:
u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.20)
riskFreeCurve = ql.FlatForward(
    0, ql.TARGET(), ql.QuoteHandle(r), ql.Actual360()
)
volatility = ql.BlackConstantVol(
    0, ql.TARGET(), ql.QuoteHandle(sigma), ql.Actual360()
)

In [5]:
process = ql.BlackScholesProcess(
    ql.QuoteHandle(u),
    ql.YieldTermStructureHandle(riskFreeCurve),
    ql.BlackVolTermStructureHandle(volatility)
)
engine = ql.AnalyticEuropeanEngine(process)
option.setPricingEngine(engine)
print(option.NPV())
print("\nGreeks:")
print(option.delta())
print(option.gamma())
print(option.vega())

4.155543462156206

Greeks:
0.5302223303784392
0.03934493301271913
20.109632428723106


## Market changes  
As I mentioned, market data are stored in Quote instances and thus can notify the option when any
of them changes. We don’t have to do anything explicitly to tell the option to recalculate: once we
set a new value to the underlying, we can simply ask the option for its NPV again and we’ll get the
updated value.

In [6]:
u.setValue(105)
print(option.NPV())

7.27556357927846


In [12]:
ql.Settings.instance().evaluationDate = today
u.setValue(105.0)
r.setValue(0.01)
sigma.setValue(0.20)
print(option.NPV())

7.295356086978629


## Other pricing methods  
The pricing-engine mechanism allows us to use different pricing methods. For comparison, I’ll first
set the input data back to what they were previously and output the Black-Scholes price.

In [13]:
model = ql.HestonModel(
    ql.HestonProcess(
        ql.YieldTermStructureHandle(riskFreeCurve),
        ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), 0, ql.Actual360())),
        ql.QuoteHandle(u),
        0.04, 0.1, 0.01, 0.05, -0.75
    )
)

engine = ql.AnalyticHestonEngine(model)
option.setPricingEngine(engine)
print(option.NPV())

7.295356086978629
